In [54]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
import itertools
import click
import numpy as np

#https://stackoverflow.com/questions/8049520/web-scraping-javascript-page-with-python
from requests_html import HTMLSession, AsyncHTMLSession

# https://www.youtube.com/watch?v=U6gbGk5WPws&list=PLzMcBGfZo4-n40rB1XaJ0ak1bemvlqumQ&index=3&ab_channel=TechWithTim
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

In [59]:
years = [str(i) for i in range(2001, 2019)] # excluding 2019, 2020, 2021 due to covid / virtual events
base_url = 'http://registration.baa.org/cfm_Archive/iframe_ArchiveSearch.cfm?mode=entry&snap=60217012&'

for year in years:
    path = 'C:\Program Files (x86)\chromedriver.exe'
    driver = webdriver.Chrome(path)
    driver.get(base_url)
    soup = BeautifulSoup(driver.page_source)
    
# give a second to load
    time.sleep(2)
    
# select year from dropdown and search
    select_year = Select(driver.find_element(by=By.NAME, value='RaceYearLowID')).select_by_visible_text(year)
    search_button = driver.find_elements(by=By.CLASS_NAME, value='submit_button')
    search_button[-1].click()
    
# grab page HTML
    soup2 = BeautifulSoup(driver.page_source, 'lxml')
    
# instantiate list for results
    results = []         
                                   
# get number of runners (to know how many pages to go to)
    runners = int(soup2.find_all('td')[2].text.split()[3])
    
# get results from current page and go on to next page
    for i in range(runners // 25):
    
        table = soup2.find("table", attrs={"class": "tablegrid_table"})
        rows = table.findAll("tr")
        
        for row in rows:
            
            a = [t.text.strip() for t in row.findAll("td")][0:]
#Don't store lines without data
            if len(a) > 0 and a != [''] and a !=['',''] and a != ['', '', '']: 
                results.append(a)
        
                
# Don't hammer the server. Give it a sec between requests.
        time.sleep(.25)
        
# next page of results
        next_button = driver.find_elements(by=By.CLASS_NAME, value='submit_button')
        next_button[-1].click()

    
    
    
# data is results but 'header rows' removed
    data = []
    for i, result in enumerate(results):
        if i%3 == 0:
            data.append(results[i] + results[i+1][1:])
            
# make data a df and save to csv    
    columns = ['year', 'bib', 'name', 'age', 'gender', 'city', 'state', 'country', 'blank', 
               'overall_place', 'gender_place', 'division_place', 'official_time', 'net_time']
    df = pd.DataFrame(data, columns=columns)
    df.to_csv(f'./Boston_Results/Original/Boston_Results_{year}.csv', index=False)

C:\Users\David\AppData\Local\Temp/ipykernel_17052/1080277195.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)
